In [16]:
from pyspark.sql import SparkSession

spark=SparkSession.builder.appName("Colab PySpark Setup").getOrCreate()

spark


#mount google drive
from google.colab import drive
from pyspark.sql.functions import year
drive.mount('/content/drive')

#read csv
sales_df=spark.read.csv('/content/drive/MyDrive/Sales_Dataset__500_Records_.csv',header=True,inferSchema=True)
#Load the CSV using Pandas, PySpark, and Dask.

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [17]:
import pandas as pd

# Load CSV with Pandas
sales_pd = pd.read_csv('/content/drive/MyDrive/Sales_Dataset__500_Records_.csv')

# Display first few rows
sales_pd.head()


,OrderID,CustomerName,ProductCategory,Amount,OrderDate,DeliveryStatus,Discount,City,PaymentMode,CustomerSince
0,2824,Donald Walker,Books,783.04,2024-12-26,Returned,0.15,Lake Joyside,Credit Card,2020-10-15
1,7912,Brandon Hall,Groceries,905.00,2024-09-12,Cancelled,0.03,New Jamesside,Wallet,2022-03-15
2,4611,Donald Booth,Fashion,657.96,2025-01-12,Returned,0.01,Lake Roberto,Wallet,2021-08-07
3,3547,Phillip Garcia,Fashion,606.89,2024-03-24,Returned,0.15,West Melanieview,Wallet,2020-08-08
4,8527,Valerie Gray,Toys,77.87,2024-08-04,Delivered,0.17,Mariastad,Cash,2022-11-15


In [18]:
!pip install dask[complete]

import dask.dataframe as dd

sales_dask = dd.read_csv('/content/drive/MyDrive/Sales_Dataset__500_Records_.csv')
sales_dask.head()  # Similar to Pandas


,OrderID,CustomerName,ProductCategory,Amount,OrderDate,DeliveryStatus,Discount,City,PaymentMode,CustomerSince
0,2824,Donald Walker,Books,783.04,2024-12-26,Returned,0.15,Lake Joyside,Credit Card,2020-10-15
1,7912,Brandon Hall,Groceries,905.00,2024-09-12,Cancelled,0.03,New Jamesside,Wallet,2022-03-15
2,4611,Donald Booth,Fashion,657.96,2025-01-12,Returned,0.01,Lake Roberto,Wallet,2021-08-07
3,3547,Phillip Garcia,Fashion,606.89,2024-03-24,Returned,0.15,West Melanieview,Wallet,2020-08-08
4,8527,Valerie Gray,Toys,77.87,2024-08-04,Delivered,0.17,Mariastad,Cash,2022-11-15


In [34]:

#2 Display the first 5 and last 5 records.
sales_df.show(5) #first 5

#last 5
# Get total count of rows
total_rows = sales_df.count()

# Subtract 5 from total count and use .collect() with .take()
last_5 = sales_df.rdd.zipWithIndex() \
            .filter(lambda row_index: row_index[1] >= total_rows - 5) \
            .map(lambda row_index: row_index[0]) \
            .collect()

# Display the last 5 rows
for row in last_5:
    print(row)


#3 Print schema and check data types.
sales_df.printSchema()

#4 Select only OrderID, CustomerName, and Amount.
sales_df.select("OrderID", "CustomerName", "Amount").show()
#5 Rename Amount to OrderAmount.
sales_df.withColumnRenamed("Amount", "OrderAmount").show()
#6 Filter orders where Amount > 500.
sales_df.filter(sales_df["Amount"] > 500).show()
#7 Filter orders from a specific city using .query() or .filter().
sales_df.filter(sales_df["City"] == "New York").show()

#8 Drop CustomerSince column.
sales_df.drop("CustomerSince").show()
#9 Add a new column FinalAmount = Amount - (Amount * Discount).
sales_df=sales_df.withColumn("FinalAmount", sales_df["Amount"] - (sales_df["Amount"] * sales_df["Discount"]))
sales_df.show()
#10 Sort by FinalAmount descending.
sales_df.orderBy(sales_df["FinalAmount"].desc()).show()
#11 Replace all “Cancelled” status with “Order Cancelled”
from pyspark.sql.functions import when
sales_df = sales_df.withColumn("DeliveryStatus",when(sales_df["DeliveryStatus"] == "Cancelled", "Order Cancelled").otherwise(sales_df["DeliveryStatus"]))

sales_df.show()
#12 Count of orders by DeliveryStatus.
sales_df.groupBy("DeliveryStatus").count().show()
#13 Average Amount by ProductCategory.
sales_df.groupBy("ProductCategory").agg({"Amount": "avg"}).show()
#14 Group by City and show total sales.
sales_df.groupBy("City").agg({"Amount": "sum"}).show()

#15 Intentionally inject nulls in City column and handle them using fillna(), dropna().
from pyspark.sql.functions import when, lit

# Inject nulls into City where City == "New York"
sales_df = sales_df.withColumn("City",when(sales_df["City"] == "New York", lit(None)).otherwise(sales_df["City"]))

sales_df.show()

#16 Use .when().otherwise() in PySpark to tag high-value customers (Amount > 800).
from pyspark.sql.functions import when

sales_df = sales_df.withColumn("CustomerCategory",when(sales_df["Amount"] > 800, "High-Value").otherwise("Low-Value"))

sales_df.show()

#17 Extract year and month from OrderDate.
from pyspark.sql.functions import year, month

sales_df.withColumn("Year", year(sales_df["OrderDate"])).withColumn("Month", month(sales_df["OrderDate"])).show()

#18 Calculate customer loyalty in years = today - CustomerSince.
from pyspark.sql.functions import current_date, datediff
sales_df.withColumn("LoyaltyYears", datediff(current_date(), sales_df["CustomerSince"]) / 365).show()

#19 Create a second DataFrame with city-wise region mapping.
city_region_df = spark.createDataFrame([
    ("New York", "Northeast"),
    ("Los Angeles", "West"),
    ("Chicago", "Midwest")
], ["City", "Region"])
#20 Perform inner and left joins with the main dataset.
sales_df.join(city_region_df, on="City", how="inner").show()
sales_df.join(city_region_df, on="City", how="left").show()
#21 Union two datasets: e.g., orders from 2023 and 2024.
sales_df_2023 = sales_df.filter(year(sales_df["OrderDate"]) == 2023)
sales_df_2024 = sales_df.filter(year(sales_df["OrderDate"]) == 2024)
sales_df_union = sales_df_2023.union(sales_df_2024)

#22 Convert each order to a JSON string and load it back into a DataFrame.
from pyspark.sql.functions import to_json, struct
sales_df.select(to_json(struct("*")).alias("order_json")).show(truncate=False)


#23 Access nested fields using explode() and get_json_object().
from pyspark.sql.functions import explode, get_json_object

# First, create a DataFrame with a nested structure for demonstration
data = [("1", '{"name": "Alice", "address": {"city": "New York", "zip": "10001"}}'),
        ("2", '{"name": "Bob", "address": {"city": "Los Angeles", "zip": "90001"}}')]

nested_df = spark.createDataFrame(data, ["id", "user_info"])

# Now, access the nested fields
nested_df.select(
    get_json_object(nested_df["user_info"], "$.name").alias("name"),
    get_json_object(nested_df["user_info"], "$.address.city").alias("city"),
    get_json_object(nested_df["user_info"], "$.address.zip").alias("zip")
).show()

# Example with explode (assuming a column with arrays)
data_with_array = [("1", ["apple", "banana", "cherry"]),
                   ("2", ["date", "elderberry"])]

array_df = spark.createDataFrame(data_with_array, ["id", "fruits"])

array_df.select(array_df["id"], explode(array_df["fruits"]).alias("fruit")).show()


#24 Create a function to tag orders: “Big”, “Medium”, “Small” based on Amount.Apply it using .apply() in Pandas, and UDF in PySpark.
def tag_order(amount):
    if amount > 1000:
        return "Big"
    elif amount > 500:
        return "Medium"
    else:
        return "Small"

# Apply tagging function
sales_pd['OrderTag'] = sales_pd['Amount'].apply(tag_order)

# View result
sales_pd[['Amount', 'OrderTag']].head()


+-------+--------------+---------------+------+----------+---------------+--------+----------------+-----------+-------------+-----------------+----------------+
|OrderID|  CustomerName|ProductCategory|Amount| OrderDate| DeliveryStatus|Discount|            City|PaymentMode|CustomerSince|      FinalAmount|CustomerCategory|
+-------+--------------+---------------+------+----------+---------------+--------+----------------+-----------+-------------+-----------------+----------------+
|   2824| Donald Walker|          Books|783.04|2024-12-26|       Returned|    0.15|    Lake Joyside|Credit Card|   2020-10-15|          665.584|       Low-Value|
|   7912|  Brandon Hall|      Groceries| 905.0|2024-09-12|Order Cancelled|    0.03|   New Jamesside|     Wallet|   2022-03-15|           877.85|      High-Value|
|   4611|  Donald Booth|        Fashion|657.96|2025-01-12|       Returned|    0.01|    Lake Roberto|     Wallet|   2021-08-07|         651.3804|       Low-Value|
|   3547|Phillip Garcia|    

,Amount,OrderTag
0,783.04,Medium
1,905.00,Medium
2,657.96,Medium
3,606.89,Medium
4,77.87,Small
